# Homework 5

## Team Members:
### 1. Dongtong Zhong (8124193969)
### 2. Liwen Dai (5282656931)
### 3. Feifan Gu (8135699631)

## Question 1

### Question 1.a

In [1]:
import pandas as pd
import numpy as np

# Load data
crsp = pd.read_csv("CRSP_Indexes.csv", parse_dates=['DATE'])
ff   = pd.read_csv("FamaFrenchDaily.csv", parse_dates=['Date'])

# Merge data at indexes where only CRSP has data
Merge_df = crsp.merge(ff, how='left', left_on='DATE', right_on='Date')
Merge_df.set_index('DATE', inplace=True)
Merge_df.drop(columns=['Date'], inplace=True)
Merge_df.head()

C:\Users\97909\AppData\Local\Temp\ipykernel_9852\3673629370.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  crsp = pd.read_csv("CRSP_Indexes.csv", parse_dates=['DATE'])


,LV,LG,SV,SG,Mkt-RF,SMB,HML,RF
DATE,,,,,,,,
2020-01-02,2050.79,2517.44,1955.12,2218.42,0.86,-0.88,-0.34,0.01
2020-01-03,2035.21,2502.53,1948.33,2214.41,-0.67,0.38,0.01,0.01
2020-01-06,2037.49,2517.99,1946.46,2223.60,0.36,-0.07,-0.55,0.01
2020-01-07,2029.00,2515.96,1939.80,2221.47,-0.19,-0.01,-0.26,0.01
2020-01-08,2034.10,2535.10,1942.71,2229.83,0.47,-0.07,-0.64,0.01


In [2]:
# Calculate excess returns for each ETFs
etf_list = ['LV', 'LG', 'SV', 'SG']
Merge_df['rf'] = Merge_df['RF'] / 100.0

for etf in etf_list:
    Merge_df[f'{etf}_Return'] = Merge_df[etf].pct_change()
    Merge_df[f'{etf}_Excess_Return'] = Merge_df[f'{etf}_Return'] - Merge_df['rf']
# Calculate the annual mean excess return, annual volatility, annual Sharpe ratio, and t-statistic for each ETF
results = {}
trading_days = 252
for etf in etf_list:
    er = Merge_df[f'{etf}_Excess_Return']      # Compute excess returns
    er = er.dropna()                           # Remove NaN values
    N  = er.count()
    mean_daily = er.mean()
    std_daily  = er.std(ddof=1)

    ann_mean = mean_daily * trading_days
    ann_std  = std_daily * np.sqrt(trading_days)
    sharpe   = ann_mean / ann_std
    t_stat   = mean_daily / (std_daily / np.sqrt(N))
    results[etf] = {
        'Annualized Mean Excess Return': ann_mean,
        'Annualized Volatility': ann_std,
        'Annualized Sharpe Ratio': sharpe,
        'T-Statistic': t_stat
    }
results_df = pd.DataFrame(results).T
results_df

,Annualized Mean Excess Return,Annualized Volatility,Annualized Sharpe Ratio,T-Statistic
LV,0.064487,0.200205,0.322106,0.753496
LG,0.165763,0.259616,0.638494,1.493615
SV,0.074011,0.264932,0.279358,0.653495
SG,0.070021,0.272320,0.257128,0.601495


### Question 1.b

In [3]:
# Calculate the portfolio return as the average return of the four ETFs, equally weighted
Merge_df['Port_Return'] = Merge_df[[f'{etf}_Return' for etf in etf_list]].mean(axis=1)
Merge_df['Port_Excess_Return'] = Merge_df['Port_Return'] - Merge_df['rf']

er_p = Merge_df['Port_Excess_Return'].dropna()
N_p  = er_p.count()
mean_daily_p = er_p.mean()
std_daily_p  = er_p.std(ddof=1)

ann_mean_p = mean_daily_p * trading_days
ann_std_p  = std_daily_p * np.sqrt(trading_days)
sharpe_p   = ann_mean_p / ann_std_p
t_stat_p   = mean_daily_p / (std_daily_p / np.sqrt(N_p))

results_port = {
    'Annualized Mean Excess Return': ann_mean_p,
    'Annualized Volatility': ann_std_p,
    'Annualized Sharpe Ratio': sharpe_p,
    'T-Statistic': t_stat_p
}

results_port_df = pd.DataFrame(results_port, index=['25%_Equal_Weighted_Portfolio'])
results_port_df

,Annualized Mean Excess Return,Annualized Volatility,Annualized Sharpe Ratio,T-Statistic
25%_Equal_Weighted_Portfolio,0.093571,0.233336,0.401012,0.938079


### Question 1.c

In [4]:
# Implement a timing strategy based on 200-day moving average for each ETF
window = 200
timing_results = {}

for etf in etf_list:
    price = Merge_df[etf]

    # 200-day moving average of price, using only past information
    ma_200 = price.rolling(window=window).mean().shift(1)   # 200-day MA up to yesterday
    lag_price = price.shift(1)                              # yesterday's price

    # Signal: 1 if yesterday's price > yesterday's 200d MA, else 0
    signal = (lag_price > ma_200).astype(int)

    # Timing strategy excess return:
    # when signal==1 we hold the ETF, otherwise we hold cash (rf).
    # Excess return vs rf is therefore signal * ETF_excess_return.
    col_name = f'{etf}_Timing_Excess_Return'
    Merge_df[col_name] = signal * Merge_df[f'{etf}_Excess_Return']

    # Drop NaNs from the warm-up period (first 200 days, etc.)
    er_t = Merge_df[col_name].dropna()
    N_t = er_t.count()

    mean_daily_t = er_t.mean()
    std_daily_t  = er_t.std(ddof=1)

    # Annualize
    ann_mean_t = mean_daily_t * trading_days
    ann_std_t  = std_daily_t * np.sqrt(trading_days)
    sharpe_t   = ann_mean_t / ann_std_t
    t_stat_t   = mean_daily_t / (std_daily_t / np.sqrt(N_t))

    timing_results[f'{etf}_Timing'] = {
        'Annualized Mean Excess Return': ann_mean_t,
        'Annualized Volatility': ann_std_t,
        'Annualized Sharpe Ratio': sharpe_t,
        'T-Statistic': t_stat_t
    }

# Collect results into a DataFrame
timing_results_df = pd.DataFrame(timing_results).T
timing_results_df


,Annualized Mean Excess Return,Annualized Volatility,Annualized Sharpe Ratio,T-Statistic
LV_Timing,0.015066,0.102326,0.147239,0.344432
LG_Timing,0.127856,0.139285,0.917942,2.147322
SV_Timing,0.043846,0.136948,0.320161,0.748946
SG_Timing,0.014894,0.140464,0.106037,0.248051


In [5]:
best_etf = timing_results_df['Annualized Sharpe Ratio'].idxmax()
print("Best ETF for the 200-day MA timing strategy (by Sharpe Ratio):", best_etf)

Best ETF for the 200-day MA timing strategy (by Sharpe Ratio): LG_Timing


Based on the result, the best ETF for the 200-day MA timing strategy in terms of Sharpe Ratio is the Large Cap Growth ETF (LG_Timing).

## <mark> Question solution summary: <mark>

In **Question 1**, we analyze the performance and risk characteristics of four style-based equity ETFs and assess whether a simple technical timing rule can improve their returns. In (a), we compute daily excess returns for the Large Value, Large Growth, Small Value, and Small Growth ETFs, and evaluate each using annualized mean excess return, volatility, Sharpe ratio, and t-statistics. In (b), we form an equal-weighted portfolio of the four ETFs and calculate its excess returns and performance metrics to assess the benefits of diversification. In (c), we apply a 200-day moving-average timing strategy to each ETF and compare the resulting risk-adjusted performance to the corresponding buy-and-hold results. Overall, Large Growth exhibits the strongest buy-and-hold performance, the equal-weighted portfolio provides moderate diversification benefits, and the moving-average rule enhances performance mainly for Large Growth while offering limited improvement for the other ETFs.

## Question 2

### Question 2.a

In [6]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import datetime

# Load data and prepare
def load_and_prepare_data():
    # Read Excel file
    df = pd.read_excel('PredictorData2024.xlsx', sheet_name='Monthly')
    
    # Convert yyyymm to date format
    df['date'] = pd.to_datetime(df['yyyymm'].astype(str), format='%Y%m')
    df.set_index('date', inplace=True)
    
    # Keep data from 1950-01-01 onwards
    df = df[df.index >= '1950-01-01']
    
    # Keep only required variables and rename
    keep_cols = ['ret', 'Rfree', 'd/p', 'tms', 'dfy', 'ntis']
    df = df[keep_cols].copy()
    df.rename(columns={'d/p': 'dp'}, inplace=True)
    
    # Calculate excess return
    df['ER'] = df['ret'] - df['Rfree']
    
    # Print results
    print(df.head())
    
    return df

# Test the function
df = load_and_prepare_data()

                 ret   Rfree        dp     tms     dfy      ntis        ER
date                                                                      
1950-01-01  0.019703  0.0009  0.067449  0.0108  0.0067  0.027102  0.018803
1950-02-01  0.019603  0.0009  0.067364  0.0102  0.0066  0.025492  0.018703
1950-03-01  0.008185  0.0010  0.067669  0.0103  0.0066  0.029291  0.007185
1950-04-01  0.045887  0.0009  0.065302  0.0099  0.0063  0.026398  0.044987
1950-05-01  0.046902  0.0010  0.063365  0.0097  0.0064  0.028572  0.045902


c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


### Question 2.b

In [7]:
# Execute rolling window regression predictions
def rolling_regression_predictions(df):
    """
    Use 10-year rolling window for regression predictions.
    The first prediction is for January 1960, based on data from
    Jan 1950 - Dec 1959.
    """
    # Create lagged variables
    df['dp_lag1'] = df['dp'].shift(1)
    df['tms_lag1'] = df['tms'].shift(1)
    df['dfy_lag1'] = df['dfy'].shift(1)
    df['ntis_lag1'] = df['ntis'].shift(1)

    # Store prediction results
    predictions = []
    prediction_dates = []

    # Set window size (10 years = 120 months)
    window_size = 120

    # Rolling window regression
    # i = window_size → 用 df.iloc[0:120]（1950-01 ~ 1959-12）预测第 120 条（1960-01）
    for i in range(window_size, len(df)):
        # Current prediction date (the month we are forecasting)
        current_date = df.index[i]

        # Extract window data (past 10 years)
        window_data = df.iloc[i-window_size:i]

        # Remove rows with NaN (due to lagged variables)
        # 理想情况下应该剩下 119 条观测（10 年减去第一个 lag 丢掉的一条）
        window_data_clean = window_data.dropna(subset=['ER', 'dp_lag1', 'tms_lag1', 'dfy_lag1', 'ntis_lag1'])

        # Ensure we have exactly 119 observations in the regression window
        if len(window_data_clean) < window_size - 1:
            continue

        # Prepare regression data
        X = window_data_clean[['dp_lag1', 'tms_lag1', 'dfy_lag1', 'ntis_lag1']]
        y = window_data_clean['ER']

        # Add constant term
        X = sm.add_constant(X)

        try:
            # Run OLS regression
            model = sm.OLS(y, X).fit()

            # Prepare predictors for forecasting:
            # use the previous month's actual predictors as input (t-1)
            last_obs = df.iloc[i-1]
            X_pred = pd.DataFrame({
                'const': [1],
                'dp_lag1': [last_obs['dp']],
                'tms_lag1': [last_obs['tms']],
                'dfy_lag1': [last_obs['dfy']],
                'ntis_lag1': [last_obs['ntis']]
            })

            # Make prediction of ER_t for current_date
            pred = model.predict(X_pred)[0]

            # Store results
            predictions.append(pred)
            prediction_dates.append(current_date)

        except Exception as e:
            print(f"Regression failed at {current_date}: {e}")
            continue

    # Create prediction results DataFrame
    results = pd.DataFrame({
        'predicted_ER': predictions
    }, index=prediction_dates)

    return results

# Main execution function
def main():
    df = load_and_prepare_data()
    print(f"Data period: {df.index.min()} to {df.index.max()}")
    print(f"Total observations: {len(df)}")

    predictions = rolling_regression_predictions(df)

    print(f"Prediction period: {predictions.index.min()} to {predictions.index.max()}")
    print(f"Total predictions: {len(predictions)}")
    print(f"First prediction value ({predictions.index[0].strftime('%Y-%m')}): "
          f"{predictions.iloc[0]['predicted_ER']:.6f}")

    # Display first few predictions
    print("\nFirst 10 predictions:")
    print(predictions.head(10))

    # Display statistical summary
    print("\nPrediction statistics summary:")
    print(predictions.describe())

    return df, predictions

# Execute main program
df, predictions = main()

c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


                 ret   Rfree        dp     tms     dfy      ntis        ER
date                                                                      
1950-01-01  0.019703  0.0009  0.067449  0.0108  0.0067  0.027102  0.018803
1950-02-01  0.019603  0.0009  0.067364  0.0102  0.0066  0.025492  0.018703
1950-03-01  0.008185  0.0010  0.067669  0.0103  0.0066  0.029291  0.007185
1950-04-01  0.045887  0.0009  0.065302  0.0099  0.0063  0.026398  0.044987
1950-05-01  0.046902  0.0010  0.063365  0.0097  0.0064  0.028572  0.045902
Data period: 1950-01-01 00:00:00 to 2024-12-01 00:00:00
Total observations: 900
Prediction period: 1960-01-01 00:00:00 to 2024-12-01 00:00:00
Total predictions: 780
First prediction value (1960-01): 0.000797

First 10 predictions:
            predicted_ER
1960-01-01      0.000797
1960-02-01     -0.001679
1960-03-01      0.001897
1960-04-01      0.006327
1960-05-01      0.009289
1960-06-01      0.009219
1960-07-01      0.019308
1960-08-01      0.015500
1960-09-01      0.0

### Question 2.c

In [8]:
def perf_stats(r, freq=12):
    """
    Compute performance statistics for a monthly excess return series.
    Output includes annualized mean, annualized volatility, Sharpe ratio,
    and the t-statistic of the mean.
    
    Parameters:
        r (Series): Monthly excess returns
        freq (int): Annualization factor (12 for monthly data)
    """
    r = r.dropna()
    N = len(r)
    mean_m = r.mean()
    std_m = r.std(ddof=1)

    ann_mean = mean_m * freq
    ann_std = std_m * np.sqrt(freq)
    sharpe = ann_mean / ann_std
    t_stat = mean_m / (std_m / np.sqrt(N))

    return {
        'Annualized Mean Excess Return': ann_mean,
        'Annualized Volatility': ann_std,
        'Annualized Sharpe Ratio': sharpe,
        'T-Statistic': t_stat
    }


def timing_strategy(df, predictions):
    """
    Construct the timing strategy using predicted excess returns.
    Weight rule:
        w_t = min(1.5, max(0.5, 100 * ER_hat_t))

    Compare:
        - Timing strategy (w_t * ER_t)
        - Buy-and-hold (ER_t)

    Returns:
        new_df      DataFrame with ER, predicted_ER, w_t, Timing_ER, BH_ER
        results_df  Performance stats for both strategies
    """
    # Merge actual ER and predicted ER
    new_df = df[['ER']].join(predictions[['predicted_ER']], how='inner')

    # Remove NaN rows (due to 10-year rolling window lags)
    new_df = new_df.dropna(subset=['ER', 'predicted_ER'])

    # Compute portfolio weight w_t = min(1.5, max(0.5, 100 × ER_hat_t))
    new_df['w_t'] = 100.0 * new_df['predicted_ER']
    new_df['w_t'] = new_df['w_t'].clip(lower=0.5, upper=1.5)

    # Timing strategy excess return
    new_df['Timing_ER'] = new_df['w_t'] * new_df['ER']

    # Buy-and-hold strategy excess return
    new_df['BH_ER'] = new_df['ER']

    # Compute performance statistics
    timing_stats = perf_stats(new_df['Timing_ER'])
    bh_stats = perf_stats(new_df['BH_ER'])

    # Create formatted comparison table
    results_df = pd.DataFrame({
        'Timing_Strategy': timing_stats,
        'Buy_and_Hold': bh_stats
    }).T

    print("\n===== Question 2.c: Timing Strategy vs Buy-and-Hold =====")
    print(results_df)

    return new_df, results_df

def main():

    new_df, results_df = timing_strategy(df, predictions)
    # return results
    return new_df, results_df

# Execute main program
new_df, results_df = main()



===== Question 2.c: Timing Strategy vs Buy-and-Hold =====
                 Annualized Mean Excess Return  Annualized Volatility  \
Timing_Strategy                       0.077941               0.140509   
Buy_and_Hold                          0.069185               0.149409   

                 Annualized Sharpe Ratio  T-Statistic  
Timing_Strategy                 0.554703     4.472161  
Buy_and_Hold                    0.463057     3.733288  


Based on the results, the timing strategy outperforms the buy-and-hold benchmark. It delivers a higher annualized mean excess return (7.79% vs. 6.92%) while also slightly reducing volatility (14.05% vs. 14.94%). Consequently, the Sharpe ratio improves from 0.46 for the buy-and-hold strategy to 0.55 for the timing strategy, and the t-statistic of the mean excess return rises from 3.73 to 4.47. Taken together, these results indicate that, over this sample period, an investor would have achieved higher and more statistically significant risk-adjusted performance by using the predictor-based timing rule rather than simply holding the market portfolio. This suggests that the timing strategy effectively capitalizes on the predictive power of the selected variables to enhance returns while managing risk.

## <mark> Question solution summary: <mark>

In **Question 2**, we examine whether common predictive variables can forecast monthly excess market returns and whether these forecasts improve portfolio performance. In (a), we load and clean the "PredictorData2024" dataset, construct the excess market return series, and retain the four predictors (dp, tms, dfy, ntis). In (b), we implement a 10-year rolling predictive regression, generating a time series of out-of-sample forecasts for excess returns starting in 1960. In (c), we use these forecasts to build a dynamic timing strategy with weights constrained between 0.5 and 1.5. Comparing this strategy with a passive buy-and-hold benchmark, we find that the timing strategy achieves higher mean excess returns, lower volatility, and a significantly higher Sharpe ratio. Overall, the predictors provide meaningful forecasting power, and using them for market timing improves performance relative to simply holding the market.